In [ ]:
# # %%capture
# !pip install ampligraph
# !pip uninstall tensorflow
# !pip install tensorflow==1.13.2

# !pip install seaborn adjustText
# !git clone https://github.com/wyldebeast-wunderliebe/incf.countryutils.git
# !pip install incf.countryutils/.

In [ ]:
%%capture 
import requests
import pandas as pd
import numpy as np
from ampligraph.evaluation import train_test_split_no_unseen 
from ampligraph.latent_features import ComplEx
import tensorflow as tf
from ampligraph.evaluation import evaluate_performance
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from incf.countryutils import transformations
from ampligraph.utils import save_model, restore_model
import os
from ampligraph.discovery import find_clusters
from sklearn.cluster import KMeans
from scipy.special import expit
%matplotlib inline

In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/zalo/data/train_kg.csv'
df = pd.read_csv(data_path)


In [ ]:
df.head()

,subj,predicate,obj
0,4,isScoreOfTeam,Chelsea
1,2,isScoreOfTeam,Stoke
2,Cahill,playerScoreFor,Chelsea
3,Willian,playerScoreFor,Chelsea
4,Crouch,playerScoreFor,Stoke


In [ ]:
triples_df = df

In [ ]:
triples = df.values.tolist()

In [ ]:
X_train, X_valid = train_test_split_no_unseen(np.array(triples), test_size=1000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (6168, 3)
Test set size:  (1000, 3)


In [ ]:
model = ComplEx(batches_count=128,
                epochs=100,
                k=100,
                eta=20,
                optimizer='adam', 
                optimizer_params={'lr':1e-2},
                loss='multiclass_nll',
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                seed=0, 
                verbose=True)

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
model.fit(X_train)

Average Loss:   0.231497: 100%|██████████| 100/100 [03:16<00:00,  1.96s/epoch]


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/zalo/data/'

/content/drive/My Drive/Colab Notebooks/zalo/data


In [ ]:
# Save the model
# model_name_path = os.path.join(data_path,'model_kg.pkl')
save_model(model, model_name_path='/content/drive/My Drive/Colab Notebooks/zalo/model_kg/model.pkl')

In [ ]:
# Restore the model
model = restore_model(model_name_path='/content/drive/My Drive/Colab Notebooks/zalo/model_kg/model.pkl')

In [ ]:
filter_triples = np.concatenate((X_train, X_valid))

In [ ]:

ranks = evaluate_performance(X_valid,
                             model=model, 
                             filter_triples=filter_triples,
                             use_default_protocol=True,
                             verbose=True)

100%|██████████| 1000/1000 [00:06<00:00, 148.54it/s]


In [ ]:


mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.52
MR: 277.72
Hits@10: 0.55
Hits@3: 0.53
Hits@1: 0.50


In [ ]:
test = np.array(['Juan Mata','playerScoreAt','90'])
scores = model.predict(test)
probs = expit(scores)
probs

array([0.95127106], dtype=float32)